In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from bs4 import BeautifulSoup
import requests, csv, string, re, datetime
from csv import writer
import time
import random

headers = { 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
cookies = dict(PHPSESSID='hu3bbnbkj9pjhm2h5ls52r5aa0', collections='audio_tech')

In [ ]:
links = [['https://web.archive.org/web/20120510091223/http://operabase.com/plan.cgi?lang=en&season=2011/12', 
 'https://web.archive.org/web/20120510091223/http://operabase.com/', '1112', 3],
['https://web.archive.org/web/20130223084132/http://www.operabase.com/plan.cgi?lang=en&season=2012/13', 
 'https://web.archive.org/web/20130223084132/http://www.operabase.com/', '1213', 1],
['https://web.archive.org/web/20140729224252/http://operabase.com:80/plan.cgi?lang=en&season=2013/14', 
            'https://web.archive.org/web/20140729224252/http://operabase.com:80/', '1314', 2],
['https://web.archive.org/web/20150924162245/http://operabase.com:80/plan.cgi?lang=en&season=2014/15', 
            'https://web.archive.org/web/20150924162245/http://operabase.com:80/', '1415', 1],
['https://web.archive.org/web/20160307032948/http://operabase.com/plan.cgi?lang=en&season=2015/16', 
            'https://web.archive.org/web/20160307032948/http://operabase.com/', '1516', 1],
['http://operabase.com/plan.cgi?lang=en&season=2016/17', 'http://operabase.com/', '1617', 1],
['http://operabase.com/plan.cgi?lang=en&season=2017/18', 'http://operabase.com/', '1718', 1],
['http://operabase.com/plan.cgi?lang=en&season=2018/19', 'http://operabase.com/', '1819', 1]]

#0,1,2,3,4,5,6,7
season_index=4

arch_url=links[season_index][0]
archbase_url=links[season_index][1]
season=links[season_index][2]
offset=links[season_index][3]

r  = requests.get(arch_url, headers=headers)
soup = BeautifulSoup(r.text, "html5lib")    
countries =[]

for l in soup.select('table a'):
    if 'plan.cgi?lang=en&season=' in l.get('href') and '&ci=':
        country = []
        country.append(l.text)
        country.append(l.get('href'))        
        countries.append(country)

listings = []
countries = countries[offset:]
processed_countries=[]
processed_keys=[]

In [ ]:
#########################################################################
#######save the pages as scraping the site is better done locally.#######
#########################################################################

import os

for i in range(100):
    for c in countries:
        url = archbase_url + c[1]
        country = c[0]
        abbr=c[1].split("=")[-1]
        if abbr not in processed_countries:
            try:
                r  = requests.get(url, headers=headers)
                soup = BeautifulSoup(r.text, "html5lib")
                #########save the country file#########
                with open("../../data/raw/operabase_html/" + season + "/" + abbr + ".html", "w") as file:
                    file.write(str(soup))   
                    
                #########theatre schedules#########
        #         time.sleep(random.random())
                schedules = soup.find_all('a')
                schedules = [s for s in schedules if 'view.cgi?lang=en&cal=' in s.get('href')]
                print
                for schedule in schedules:
                    cal_url = schedule.get('href')
                    filename="../../data/raw/operabase_html/" + season + "/" + cal_url.strip() + ".html"
                    if os.path.exists(filename)==False:
#                         print
                        if cal_url.split("=")[-1] not in processed_keys:                       
#                             time.sleep(2*random.random())
                            r  = requests.get(archbase_url + cal_url, headers=headers)
                            cal_soup = BeautifulSoup(r.text, "html5lib")            

                            with open("../../data/raw/operabase_html/" + season + "/" + cal_url.strip() + ".html", "w") as file:
                                file.write(str(cal_soup))                        
                            processed_keys.append(cal_url.split("=")[-1])                
            #         print(schedules)            
                processed_countries.append(abbr)
            except:
                None

In [22]:
import glob
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

seasons=['1112','1213','1314','1415','1516','1617','1718','1819']
country_dict = { 
    'al':'Albania', 'am':'Armenia', 'at':'Austria', 'az':'Azerbaijan', 'ba':'Bosnia-Herzegovina', 'be':'Belgium',
    'bg':'Bulgaria', 'by':'Belarus', 'ca':'Canada', 'ch':'Switzerland', 'cl':'Chile','cz':'Czech Republic',
    'de':'Germany', 'dk':'Denmark', 'ee':'Estonia', 'eg':'Egypt', 'es':'Spain', 'fi':'Finland', 'fr':'France',
    'ge':'Georgia', 'gr':'Greece', 'hr':'Croatia', 'hu':'Hungary', 'ie':'Ireland', 'il':'Israel', 'is':'Iceland',
    'it':'Italy', 'jp':'Japan', 'kg':'Kyrgyzstan', 'kr':'South Korea', 'kz':'Kazakhstan', 'lt':'Lithuania',
    'lu':'Luxembourg', 'lv':'Latvia', 'mc':'Monaco', 'md':'Moldova', 'mk':'Macedonia', 'mn':'Mongolia', 'mt':'Malta',
    'mx':'Mexico', 'nl':'Netherlands', 'no':'Norway', 'om':'Oman', 'pe':'Peru', 'pl':'Poland', 'pt':'Portugal', 
    'ro':'Romania', 'rs':'Serbia', 'ru':'Russia', 'se':'Sweden', 'sg':'Singapore', 'si':'Slovenia', 'sk':'Slovakia', 
    'sr':'Serbia',  'th':'Thailand', 'tr':'Turkey', 'ua':'Ukraine', 'uk':'United Kingdom', 'us':'United States', 
    'uz':'Uzbekistan', 'za':'South Africa'
}
# seasons=['1112']

listings=[]

for season in seasons:
    files = glob.glob("../../data/raw/operabase_html/" + season + "/" + '*.html')
    files = [f for f in files if 'lang' not in f]
#     files= ['../../data/raw/operabase_html/1112/at.html']
           
    print('Season: ' + season)
#     print(files)
    for file in files:
        
        if country_dict.get(file[:-5].split('/')[-1]):
            country = country_dict.get(file[:-5].split('/')[-1])
        else:
            print('Missing key: ' + file[:-5].split('/')[-1])        
        
        with open(file, encoding="utf-8") as f:
            data = f.read()
            soup = BeautifulSoup(data, 'html.parser')            
        schedules = soup.find_all('a')
        schedules = [s for s in schedules if 'view.cgi?lang=en&cal=' in s.get('href')]        
        
        for idx,schedule in enumerate(schedules):
            map_dict = {}        
            try:
                items = schedule.next_sibling.next_sibling.find_all('a')
                par_item = schedule.next_sibling.next_sibling                               
                    
                for item in items:
                    work = item.get_text()
                    composer = re.search('\((.*?)\)', item.parent.next_sibling).group(0)[1:-1]
                    map_dict[work]=composer          

            except:
                None

            city = schedule.previous_sibling.previous_sibling.find('b').text
            theatre = ','.join(schedule.previous_sibling.previous_sibling.text.split(',')[1:]).strip()        
            url_cal = "../../data/raw/operabase_html/" + season + "/" + schedule.get('href') + '.html'
            
            try:                
                with open(url_cal, encoding="utf-8") as f:
                    data_cal = f.read()
                    soup_cal = BeautifulSoup(data_cal, 'html.parser')

                shows = soup_cal.find_all('b')[8:]
                for show in shows:
                    info = []
                    info.append(country)
                    info.append(city)
                    info.append(theatre)                            
                    info.append(show.text)
                    link = show.parent.parent.parent.find('a').get('href')
                    info.append(link.split("date=")[-1])                
                    try:
                        composer_key = process.extract(show.text, list(map_dict.keys()), limit=1)
                        composer_key = composer_key[0][0]
                        composer = map_dict.get(composer_key)
                    except:
                        composer = ''                
                    info.append(composer)
                    listings.append(info)        
            except:
                print(url_cal)
                    

import pandas as pd                     
headers = ['country', 'city', 'theatre', 'work', 'date', 'composer']
df = pd.DataFrame(listings, columns=headers)
df.to_csv('../../data/processed/listings/operabase_2.csv')

Season: 1112
Season: 1213
Season: 1314
Season: 1415
Season: 1516
Season: 1617
Season: 1718
Season: 1819
